In [1]:
import sys
from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 3 -p
%matplotlib inline


## imports

In [108]:
from pydantic import BaseModel, RootModel
from devtools import pprint

In [109]:
from ocrorchestrator.config.app_config import AppConfig

## testing config

In [34]:
config_data = {
        "general": {
            # General config fields
        },
        "categories": {
            "proof_of_funds": {
                "validation": {
                    "processor": "llm",
                    "handler": "gemini-pro-vision"
                },
                "extraction": {
                    "processor": "custom",
                    "handler": "ProofOfFundsExtractor"
                }
            },
            "checks": {
                "extraction": {
                    "processor": "custom",
                    "handler": "ProofOfFundsExtractor"
                }
            }
        }
    }

In [35]:
mm = AppConfig(**config_data)

In [14]:
mm.categories["proof_of_funds"]

NameError: name 'mm' is not defined

In [15]:
for cat, tsk, cfg in mm.iterate():
    print(cat, tsk, cfg)

NameError: name 'mm' is not defined

In [16]:
pprint(mm.get_task_config("proof_of_funds", "validation"))

Pretty printing has been turned OFF


In [115]:
from ocrorchestrator.utils.misc import create_dynamic_message_model

In [116]:
dyn = create_dynamic_message_model(["field1", "field2"])

In [117]:
dyn_obj = dyn(**{"field1": "sdada", "field2": "daad"})

In [118]:
pprint(dyn_obj)

DynamicMessage(
    field1='sdada',
    field2='daad',
)


## testing api

In [61]:
from ocrorchestrator.main import app

In [62]:
from fastapi.testclient import TestClient
from ocrorchestrator.config.app_config import AppConfig

In [63]:
client = TestClient(app)

HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/b7bfa10f-f6da-4075-8e76-aee7867cc155 "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/6f5eeb76-cb93-4387-9e64-38e949e4608b "HTTP/1.1 404 Not Found"


In [64]:
from base64 import b64encode
from pathlib import Path

b64 = b64encode(Path('../data/images/arch.png').open("rb").read())

HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/dd253886-bf2e-44af-a941-af7b3c5ac0cb "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/b7bfa10f-f6da-4075-8e76-aee7867cc155 "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/dd253886-bf2e-44af-a941-af7b3c5ac0cb "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/6f5eeb76-cb93-4387-9e64-38e949e4608b "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/6f5eeb76-cb93-4387-9e64-38e949e4608b "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/b7bfa10f-f6da-4075-8e76-aee7867cc155 "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/dd253886-bf2e-44af-a941-af7b3c5ac0cb "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/6f5eeb76-cb93-4387-9e64-38e949e4608b "

In [65]:
req_body = {
    "image": b64.decode(),
    "category": "proof_of_funds",
    "task": "extraction",
    "guid": "dummyguid",
}

In [66]:
# from gradio_client import file
# result = clnt.predict(
# 		file('../data/images/arch.png'),	# filepath in 'Image' Image component
# 		"ocr \nReturn only the shapes as JSON",	# str in 'Prompt' Textbox component
#         "paligemma-3b-mix-224",
# 		"greedy",	# Literal['greedy', 'nucleus(0.1)', 'nucleus(0.3)', 'temperature(0.5)'] in 'Decoding' Dropdown component
# 		api_name="/compute"
# )



In [67]:
# print(result[0]['value'][0]['token'])

HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/dd253886-bf2e-44af-a941-af7b3c5ac0cb "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/b7bfa10f-f6da-4075-8e76-aee7867cc155 "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/6f5eeb76-cb93-4387-9e64-38e949e4608b "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/dd253886-bf2e-44af-a941-af7b3c5ac0cb "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/6f5eeb76-cb93-4387-9e64-38e949e4608b "HTTP/1.1 404 Not Found"


In [68]:
with TestClient(app) as client:
    response = client.post("/predict", json=req_body)
    print(response.json())

{"event": "Running LLM setup", "level": "info", "func_name": "setup", "lineno": 27, "timestamp": "2024-06-30T15:31:16.165814Z"}
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/dd253886-bf2e-44af-a941-af7b3c5ac0cb "HTTP/1.1 404 Not Found"
Loaded as API: https://big-vision-paligemma.hf.space ✔
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/6f5eeb76-cb93-4387-9e64-38e949e4608b "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/config "HTTP/1.1 200 OK"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/dd253886-bf2e-44af-a941-af7b3c5ac0cb "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/6f5eeb76-cb93-4387-9e64-38e949e4608b "HTTP/1.1 404 Not Found"
HTTP Request: GET https://big-vision-paligemma.hf.space/info?serialize=False "HTTP/1.1 200 OK"
HTTP Request: GET https://big-vision-paligemma.hf.space/heartbeat/b7bfa10f-f6da-4075-8e76-aee7867cc155 "HTTP/1.1 404 Not Found"
H

In [29]:
test_()

{"event": "Running LLM setup", "level": "info", "func_name": "setup", "lineno": 27, "timestamp": "2024-06-30T13:53:19.639017Z"}
{"event": "Running LLM processor", "guid": "dummyguid", "api_name": "/predict", "trans_id": "75b0e676-876b-4223-bdf8-2cea5758d3d0", "client_host": "testclient", "method": "POST", "level": "info", "func_name": "process", "lineno": 31, "timestamp": "2024-06-30T13:53:19.642351Z"}
{"event": "OK", "guid": "dummyguid", "api_name": "/predict", "status_code": 200, "trans_id": "75b0e676-876b-4223-bdf8-2cea5758d3d0", "client_host": "testclient", "method": "POST", "level": "info", "func_name": "dispatch", "lineno": 48, "timestamp": "2024-06-30T13:53:19.643861Z"}
HTTP Request: POST http://testserver/predict "HTTP/1.1 200 OK"
{'status': 'OK', 'status_code': 200, 'message': {'name': 'John Doe', 'invoice_no': 'XYZ123'}}


In [64]:
from ocrorchestrator.main import LocalRepo, config_path, repo_args

In [73]:
config_path, repo_args

('configs/config_v1.yaml', ['/home/ajkdrag/workspace/projects/ocrorchestrator/data/my-bucket'])

In [74]:
repo = LocalRepo(*repo_args)

In [75]:
repo.get_yaml(config_path)

{'general': None, 'categories': {'proof_of_funds': {'validation': {'processor': 'llm', 'handler': 'gemini-pro-vision'}, 'extraction': {'processor': 'custom', 'handler': 'ProofOfFundsExtractor'}}, 'checks': {'extraction': {'processor': 'custom', 'handler': 'ProofOfFundsExtractor'}}}}

In [76]:
cfg = AppConfig(**repo.get_yaml(config_path))

In [77]:
cfg

AppConfig(general=GeneralConfig(prompts_dir='prompt_templates', models_dir='models'), categories={'proof_of_funds': OrderedDict([('validation', TaskConfig(processor='llm', handler='gemini-pro-vision', model=None, prompt_template=None, extra_kwargs={}, fields=None, classes=None)), ('extraction', TaskConfig(processor='custom', handler='ProofOfFundsExtractor', model=None, prompt_template=None, extra_kwargs={}, fields=None, classes=None))]), 'checks': OrderedDict([('extraction', TaskConfig(processor='custom', handler='ProofOfFundsExtractor', model=None, prompt_template=None, extra_kwargs={}, fields=None, classes=None))])})

In [78]:
cfg.general.prompts_dir

'prompt_templates'

In [79]:
cfg.general.models_dir

'models'

## testing dyn msg

In [11]:
class OCRResponse(BaseModel):
    status: str
    status_code: int
    message: Any

In [12]:
rsp = OCRResponse(**{
    "status": "OK",
    "status_code": 200,
    "junk_attr": 32,
    "message": {
        "field1": 2323,
        "field2": "sadada",
    }
})

In [14]:
rsp

OCRResponse(status='OK', status_code=200, message={'field1': 2323, 'field2': 'sadada'})

In [19]:
from ocrorchestrator.datamodels.api_io import OCRException

Reloading 'ocrorchestrator.datamodels.api_io'.


In [20]:
exc = OCRException(ErrorCode.PROCESSING_ERROR)

In [23]:
from fastapi.exception_handlers import http_exception_handler

In [26]:
from ocrorchestrator.routers import ocr_router

In [31]:
ErrorCode.PROCESSING_ERROR.name

Reloading 'ocrorchestrator.utils.constants'.


'PROCESSING_ERROR'